In [1]:
import pandas as pd 

In [12]:
rankings = pd.read_csv('ticker_ranking.csv')
tickers = pd.read_csv('tickers.csv')

In [16]:
tickers.head()

,Symbol,Name
0,42,42 Coin
1,300,300 token
2,365,365Coin
3,404,404Coin
4,433,433 Token


In [2]:
from fuzzywuzzy import fuzz
import operator
import itertools


In [23]:
score_threshold = 85

combinations = list(itertools.product(rankings['Currency Name'].values.ravel(), tickers['Name'].values.ravel()))
scores = {x: fuzz.ratio(x[0], x[1]) for x in combinations}
matches = [max({k:v for k,v in scores.items() if k[0] == x}.items(), key=operator.itemgetter(1)) for x in rankings['Currency Name'].values.ravel()]
matches = {x[0][0]:x[0][1] for x in matches if x[1] >= score_threshold}

In [29]:
not_found = list(set(rankings['Currency Name'].values.ravel()) - set(matches.keys()))
not_found = {x:x for x in not_found}

matches = dict(**matches, **not_found)

In [30]:
rankings['Currency Name'] = rankings['Currency Name'].apply(lambda x: matches[x])

In [36]:
ticker_dict = {x[0]:x[1] for x in tickers[['Name', 'Symbol']].values}

In [49]:
rankings['Ticker'] = rankings['Currency Name'].apply(lambda x: ticker_dict[x] if x in ticker_dict.keys() else "NOT_FOUND")

In [51]:
rankings.to_csv('ticker_ranking.csv', index = False)

In [2]:
import binance_api

In [3]:
api = binance_api.Binance("config.yaml")

In [4]:
api.get_binance_data("BTCUSDC", "1d", "08 Feb 2021", "12 Feb 2021")

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2021-02-08,38860.36000000,47500.00000000,37000.00000000,46452.01000000,4452.16061300,1612828799999,189569359.35988677,81315,2484.57905800,106072171.68883256,0
1,2021-02-09,46457.14000000,48216.56000000,45000.00000000,46488.27000000,4025.38658900,1612915199999,187837003.25450567,75108,2240.67414800,104600014.11159628,0
2,2021-02-10,46480.68000000,47379.74000000,43741.91000000,44858.18000000,3472.69011600,1613001599999,158239756.02866267,68657,1882.16539400,85740614.95112431,0
3,2021-02-11,44844.88000000,48692.99000000,44007.78000000,47997.30000000,2685.78931500,1613087999999,126042879.34257909,51625,1420.73495500,66757929.03364827,0
4,2021-02-12,48007.40000000,48999.00000000,46191.22000000,47446.08000000,3020.94994800,1613174399999,143659169.96995944,55955,1626.35086600,77407789.42839803,0


In [5]:
teste = {'btc' : 0.2, "eth" : 0.3, "kkk" : 0.4}

In [16]:
pd.DataFrame.from_dict(teste, orient = 'index',columns = [ 'volume']).reset_index()

,index,volume
0,btc,0.2
1,eth,0.3
2,kkk,0.4


In [2]:
import get_top_cripto

In [3]:
result = get_top_cripto.find_top_cripto()

 3.30it/s]could get data from GOLOS
100%|██████████| 542/542 [02:59<00:00,  3.01it/s]could get data from ZRX



In [9]:
import alphavantage